In [5]:
import json, os
import langgraph, langchain_openai, langchain_core
from dotenv import load_dotenv
from tqdm import tqdm
import mlflow

load_dotenv()

class clean_call_test:
    def __init__(self, llm, graph_file):

        self.llm = llm    

        with open(graph_file, 'r', encoding='utf-8') as f:
            self.graph_data = json.load(f)
            self.node = self.graph_data['nodes']

    def test_clean_call(self):
        # Initialize or load existing results
        try:
            with open('clean_call_results.json', 'r', encoding='utf-8') as f:
                all_results = json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            all_results = {"clean_call_results": []}

        for node in tqdm(self.node, desc="Testing nodes clean call"):
            node_clean_call_results = []
            
            output = self.llm.invoke(node['input'])
        
            # Convert AIMessage to string if needed
            if hasattr(output, 'content'):
                output_str = output.content
            else:
                output_str = str(output)

            result = {
                "output": output_str
            }

            node_clean_call_results.append(result)

                # Save results after each jailbreak attempt
            node_result = {
                "id": node['id'],
                "clean_call": node_clean_call_results
            }
                
            all_results["clean_call_results"].append(node_result)
                
            # Write to file
            with open('clean_call_results.json', 'w', encoding='utf-8') as f:
                json.dump(all_results, f, indent=2)        

In [6]:
llm = langchain_openai.ChatOpenAI(
                    model="deepseek/deepseek-chat",
                    openai_api_key=os.getenv("DEEPSEEK_OPENROUTER_API_KEY"),
                    openai_api_base="https://openrouter.ai/api/v1",
                    temperature=0.2
                )

clean_call_test = clean_call_test(llm, '../data/agentic_graph.json')

In [7]:
mlflow.langchain.autolog()
mlflow.set_experiment("clean_call")
mlflow.set_tracking_uri("http://localhost:5000")

clean_call_test.test_clean_call()

2025/05/15 09:17:01 INFO mlflow.tracking.fluent: Experiment with name 'clean_call' does not exist. Creating a new experiment.
Testing nodes clean call: 100%|██████████| 16/16 [08:32<00:00, 32.02s/it]


[Trace(request_id=d16cc362d7884a7784d399ae17ad0f1c), Trace(request_id=4429f6ebb84c4a54acddd87026e1ee7c), Trace(request_id=1848b7adeef048faa0d82a8fbb2d3ac1), Trace(request_id=c3b9fb91272b4e729735f2d2b890dfa5), Trace(request_id=769aef905b9d49c8a8458aa18934bfdf), Trace(request_id=feab8010dc4849a480410debfe5847bc), Trace(request_id=10aebc03ffa744d19f3747571d24b909), Trace(request_id=6ba9b2f0a6df4b9db13df9925938ac31), Trace(request_id=26a5588cbaed4fd8b83f321f82c1dda0), Trace(request_id=af37e46bb9954b4186bbb53e754abafd)]